In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -r /content/sample_data
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/ydrm75xywg-1.zip
!unzip /content/ydrm75xywg-1.zip
!mkdir /content/dataset
!unzip /content/ydrm75xywg-1/"Stenosis detection".zip -d /content/dataset
!mv /content/dataset/dataset /content/dataset/images
!rm -r /content/dataset/video_test
!rm -r /content/dataset/video_val
!rm /content/ydrm75xywg-1.zip
!rm -r /content/ydrm75xywg-1
!pip install ultralytics

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/dataset/14_066_10_0074.bmp  
  inflating: /content/dataset/dataset/14_066_9_0061.bmp  
  inflating: /content/dataset/dataset/14_023_2_0102.bmp  
  inflating: /content/dataset/dataset/14_048_2_0073.bmp  
  inflating: /content/dataset/dataset/14_073_1_0047.bmp  
  inflating: /content/dataset/dataset/14_015_1_0024.bmp  
  inflating: /content/dataset/dataset/14_096_2_0055.xml  
  inflating: /content/dataset/dataset/14_087_3_0047.xml  
  inflating: /content/dataset/dataset/14_087_3_0051.xml  
  inflating: /content/dataset/dataset/14_096_2_0011.xml  
  inflating: /content/dataset/dataset/14_050_6_0030.xml  
  inflating: /content/dataset/dataset/14_051_5_0026.xml  
  inflating: /content/dataset/dataset/14_087_3_0064.xml  
  inflating: /content/dataset/dataset/14_084_9_0044.xml  
  inflating: /content/dataset/dataset/14_096_5_0036.xml  
  inflating: /content/dataset/dataset/14_051_2_0053.xml  
  inflating: /conten

In [3]:
cd /content

/content


In [4]:
!git clone https://github.com/amirrezarajabi/coronary-artery-stenosis-detection.git

Cloning into 'coronary-artery-stenosis-detection'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 3), reused 12 (delta 2), pack-reused 0
Receiving objects: 100% (13/13), 4.48 KiB | 4.48 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [5]:
!python /content/coronary-artery-stenosis-detection/partition_dataset.py -x -i /content/dataset/images -o /content/dataset -r 0.1

/content/coronary-artery-stenosis-detection/partition_dataset.py:35: DeprecationWarning: Flags not at the start of the expression '([a-zA-Z0-9\\s_\\\\.\\-\\' (truncated) but at position 27
  if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png|.bmp)$', f)]


In [6]:
!rm -rf /content/dataset/images
!mkdir /content/dataset/images
!mkdir /content/dataset/labels
!mkdir /content/dataset/labels/train
!mkdir /content/dataset/labels/test
!mv /content/dataset/train /content/dataset/images
!mv /content/dataset/test /content/dataset/images

In [7]:
import os
import glob
from xml.etree import ElementTree as ET
from PIL import Image
from tqdm import tqdm

def make_label(img_path):
    Image.open(img_path).save(img_path.replace("bmp", "jpg"))
    xml_path = img_path.replace("bmp", "xml")
    txt_path = img_path.replace("bmp", "txt").replace("images", "labels")
    with open(xml_path, 'r') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        size = root.find('size')
        W = int(size.find('width').text)
        H = int(size.find('height').text)
        boxes = []
        for obj in root.findall('object'):
            class_label = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            x_center = (xmin + xmax) / 2.0 / W
            y_center = (ymin + ymax) / 2.0 / H
            width = (xmax - xmin) / W
            height = (ymax - ymin) / H
            boxes.append(f"0 {x_center} {y_center} {width} {height}")
    with open(txt_path, "w") as f:
        f.write('\n'.join(boxes))
    os.remove(xml_path)
    os.remove(img_path)

In [8]:
train_images = glob.glob("/content/dataset/images/train/*.bmp")
for ti in tqdm(train_images):
  make_label(ti)

test_images = glob.glob("/content/dataset/images/test/*.bmp")
for ti in tqdm(test_images):
  make_label(ti)

100%|██████████| 833/833 [00:04<00:00, 205.09it/s]


In [9]:
!mkdir /content/drive/MyDrive/yolo_ws

In [10]:
cd /content/drive/MyDrive/yolo_ws

/content/drive/MyDrive/yolo_ws


In [11]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/coronary-artery-stenosis-detection/dataset.yaml", epochs=100, imgsz=640, batch=128)

100%|██████████| 6.23M/6.23M [00:00<00:00, 253MB/s]


Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/coronary-artery-stenosis-detection/dataset.yaml, epochs=100, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 755k/755k [00:00<00:00, 97.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


...

    100/100      16.9G      1.369     0.5421      1.096         67        640: 100%|██████████| 59/59 [00:42<00:00,  1.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]

                   all        833        833       0.97      0.971      0.986      0.503



100 epochs completed in 1.485 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


                   all        833        833      0.975      0.966      0.984      0.507
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train
